In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
from simulation import get_setup

distance = 15
azimuths = [0, 15, 30]

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
source, mic_positions = get_setup(distance_cm=distance, azimuth_deg=azimuths[0])
[ax.scatter(*mic[:2], label=f'mic{i}, $\\theta=${azimuths[0]}$^\\circ$', marker='x') for i, mic in enumerate(mic_positions)]
for azimuth_deg in azimuths[1:]:
    source, mic_positions = get_setup(distance_cm=distance, azimuth_deg=azimuth_deg, ax=ax)
ax.set_title(f"wall distance $d=${distance}cm and angle $\\theta$={azimuths}deg")
handles, labels = ax.get_legend_handles_labels()
h_mics = {l: h for h, l in zip(handles, labels) if '\\theta' in l}
l1 = ax.legend(h_mics.values(), h_mics.keys(),loc="lower right", bbox_to_anchor=[.5, 0])

h_other = {l: h for h, l in zip(handles, labels) if not 'mic' in l}
ax.legend(h_other.values(), h_other.keys(),loc="lower left", bbox_to_anchor=[.5, 0])
ax.add_artist(l1)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
fig.savefig(f'plots/setup.pdf', bbox_inches='tight')

# Experimental distance-frequency matrix

In [ ]:
# exp_name = '2021_02_09_wall_tukey';
# exp_name = '2021_02_09_wall';

exp_name = "2021_02_23_wall";  # old buzzer
#exp_name = "2021_02_25_wall"; # new buzzer
#exp_name = "2021_03_01_flying"
# exp_name = '2020_12_9_rotating';
# exp_name = '2020_11_26_wall';
# fname = f'results/{exp_name}_real.pkl'

mic_type = "audio_deck"
motors = 0 #"all45000"

In [ ]:
from wall_detector import WallDetector
from generate_df_results import wall_detector_from_df

wall_detector = WallDetector()
backup_exists = wall_detector.fill_from_backup(exp_name, mic_type, motors)
if not backup_exists: 
    print('generate data using script generate_df_results.')

In [ ]:
wall_detector.df

# DF analysis

In [ ]:
from wall_detector import prune_df_matrix

df_matrix_raw, df_dist, df_freq_raw = wall_detector.get_df_matrix()
df_matrix, df_freq, indices = prune_df_matrix(
    df_matrix_raw, df_freq_raw, verbose=True
)

In [ ]:
from plotting_tools import add_colorbar

fig, ax_all = plt.subplots()
for mic_idx in range(4):
    fig, ax = plt.subplots()
    im = ax.pcolormesh(df_dist, df_freq_raw, df_matrix_raw[mic_idx])
    ax.set_title(f"mic{mic_idx} original")
    add_colorbar(fig, ax, im)
    fig, ax = plt.subplots()
    im = ax.pcolormesh(df_dist, df_freq, df_matrix[mic_idx])
    ax.set_title(f"mic{mic_idx} pruned")
    ax_all.plot(df_freq, np.nanmean(df_matrix[mic_idx], axis=1), label=f'mic{mic_idx}')
    add_colorbar(fig, ax, im)

# 1. study distance slices

In [ ]:
from simulation import get_dist_slice_theory
from copy import deepcopy
from pandas_utils import fill_nans

def plot_ffts(slice_exp, slice_the, dist):
    fig_f, axs_f = plt.subplots(1, slice_exp.shape[0], squeeze=False, sharey=True)
    fig_f.set_size_inches(10, 5)
    fig_f.suptitle(f"FFT of standardized distance slices, at frequency {f:.0f}Hz")
    for m in range(slice_exp.shape[0]):
        slice_exp_norm = deepcopy(slice_exp[m])
        slice_the_norm = deepcopy(slice_the[m])
        slice_exp_norm -= np.mean(slice_exp_norm)
        slice_the_norm -= np.mean(slice_the_norm)

        n = max(len(slice_exp_norm), 1000)
        freqs = np.fft.rfftfreq(n, d=dist[1] - dist[0])  # unit: 1/cm
        fft_exp = np.abs(np.fft.rfft(slice_exp_norm, n=n))
        fft_exp /= np.sum(fft_exp)
        fft_theory = np.abs(np.fft.rfft(slice_the_norm, n=n))
        fft_theory /= np.sum(fft_theory)

        axs_f[0, m].plot(freqs, fft_theory, label="theoretical")
        axs_f[0, m].plot(freqs, fft_exp, label="measured")
        axs_f[0, m].set_title(f"mic{m}")
        axs_f[0, m].legend(loc="upper right")
    return fig_f

valid_freqs = df_freq
for i, f in enumerate(df_freq):
    slice_exp = df_matrix[:, i, :]
    if np.any(np.isnan(slice_exp)):
        slice_exp = fill_nans(slice_exp, df_dist)
    slice_the = get_dist_slice_theory(f, df_dist).T
    
    fig_f = plot_ffts(
        slice_exp=slice_exp,
        slice_the=slice_the,
        dist=df_dist,
    )
    break

In [ ]:
from simulation import factor_distance_to_delta

for mic in range(4):
    print('mic', mic)
    for distance in np.arange(10, 50, step=10):
        f = factor_distance_to_delta(distance, mic)
        print(distance, f)

In [ ]:
# distance slice algorithm
from inference import get_approach_angle_fft, get_gamma_distribution
from plotting_tools import plot_performance, save_fig
from estimators import get_estimate

EPS = 1e-10
gt_gamma = 90 # in degrees
err_dict = {f"mic{m}": [np.nan]*len(df_freq) for m in range(df_matrix.shape[0])}
for i, f in enumerate(df_freq):
    
    d_slices = fill_nans(df_matrix[:, i, :], df_dist)
    
    fig, ax = plt.subplots()
    fig.set_size_inches(7, 5)
    for mic_idx in range(df_matrix.shape[0]):
        d_slice = d_slices[mic_idx]
        ratios, prob = get_approach_angle_fft(d_slice, f, df_dist,
                                             n_max=1000, bayes=False, reduced=False)
        gammas, prob = get_gamma_distribution(ratios, prob, factor=1.9)
        gamma = get_estimate(gammas, prob)
        
        ax.plot(gammas, prob)
        ax.axvline(gamma, label=f'mic{mic_idx}: $\\gamma$={gamma:.1f}', color=f'C{mic_idx}')
        err_dict[f"mic{mic_idx}"][i] = gamma - gt_gamma
        
    ax.set_title(f'frequency {f:.0f}Hz')
    ax.set_xlabel('angle of approach $\\gamma$ [deg]')
    ax.set_ylabel('probability')
    ax.legend()
        
fname = f"{exp_name}_{motors}"
fig, axs = plot_performance(err_dict, xs=df_freq, 
           xlabel="frequency [Hz]", ylabel="error [deg]")
save_fig(fig, f'plots/{fname}_distance_slice_performance.pdf')

# 2. study full matrices

In [ ]:
from calibration import get_calibration_function_fit
from calibration import get_calibration_function_median

fig, ax = plt.subplots()
calib_function_median = get_calibration_function_median(exp_name, mic_type, ax=ax)
ax.set_title("median")

fig, ax = plt.subplots()
calib_function_median_one = get_calibration_function_median(
    exp_name, mic_type, ax=ax, fit_one_gain=True
)
ax.set_title("median one")

fig, ax = plt.subplots()
calib_function_fit = get_calibration_function_fit(exp_name, mic_type, ax=ax)
ax.set_title("fit")

fig, ax = plt.subplots()
calib_function_fit_one = get_calibration_function_fit(
    exp_name, mic_type, ax=ax, fit_one_gain=True
)
ax.set_title("fit one")

In [ ]:
from simulation import get_df_theory
from wall_detector import normalize_df_matrix

min_value = np.inf
max_value = -np.inf

chosen_mics = range(4)

distances_grid = np.arange(min(df_dist), max(df_dist) + 1)
distances_idx = np.argmin(np.abs(distances_grid[:, None] - df_dist[None, :]), axis=0)

results = pd.DataFrame(columns=["normalization", "matrix", "values"])
method_dict = {
    'raw': "",
    'theoretical': "",
    'calibration-median': calib_function_median,
    'calibration-median-one': calib_function_median_one,
    'calibration-fit': calib_function_fit,
    'calibration-fit-one': calib_function_fit_one,
    #'calibration-online': "calibration-online",
    #'standardize': "standardize",
    #'zero_mean': "zero_mean",
    #'normalize': "normalize",
    #'calibration-offline-old': calib_function_old,
}

df_theory_pruned = get_df_theory(df_freq, df_dist, chosen_mics=chosen_mics)

for j, (key, method) in enumerate(method_dict.items()):
    values = None
    if key == "raw":
        df_norm = deepcopy(df_matrix)
    elif key == "theoretical":
        df_norm = deepcopy(df_theory_pruned[:, :, distances_idx])
    else:
        df_norm, values = normalize_df_matrix(
            df_matrix=df_matrix, freqs=df_freq, method=method
        )
    results.loc[len(results), :] = {
        "normalization": key,
        "matrix": df_norm,
        "values": values,
    }
    # print(key, data_type, df_norm.shape)
    min_value = min(min_value, -abs(np.min(df_norm)))
    max_value = max(max_value, abs(np.max(df_norm)))
print('done')

In [ ]:
from plotting_tools import plot_df_matrix, save_fig

min_freq = min(df_freq)
max_freq = max(df_freq)

min_value = None
max_value = None

fname = f"{exp_name}_{motors}"

for normalization, df in results.groupby("normalization", sort=False):
    matrix = df.iloc[0].matrix
    n_mics = matrix.shape[0]
    fig, axs = plt.subplots(1, n_mics, sharey=True)
    fig.set_size_inches(n_mics*5, 5)
    axs[0].set_ylabel('frequency [Hz]')
    for i in range(n_mics):
        df_exp = df.iloc[0].matrix[i]
        ax, im = plot_df_matrix(
            df_dist,
            df_freq,
            df_exp,
            ax=axs[i],
            min_freq=min_freq,
            max_freq=max_freq,
            vmin=min_value,
            vmax=max_value,
        )
        ax.set_title(f"mic{i} {normalization}")
        ax.set_xlabel('distance [cm]')
    add_colorbar(fig, ax, im)
    save_fig(fig, f"plots/{fname}_matrices_{normalization}.png")

# 3. Evaluate algorithm performance

In [ ]:
normalize_method = "calibration-fit"
method = method_dict[normalize_method]
matrix = results.loc[results.normalization == normalize_method].iloc[0].matrix

distance = 10
mic_idx = 1
freq_slice = matrix[mic_idx, :, distance]
plt.figure()
plt.plot(df_freq, freq_slice)

noisy_freq = df_freq + np.random.normal(scale=20, size=len(df_freq))
freq_slice_raw, used_freq, stds = wall_detector.get_frequency_slice_fixed(
    noisy_freq, distance=distance, normalize_method=method, mics=[mic_idx], verbose=True
)
plt.plot(used_freq, freq_slice_raw[0])

In [ ]:
from inference import get_probability_cost, get_probability_bayes
from simulation import get_freq_slice_theory
from estimators import DistanceEstimator, get_estimate
import progressbar

err_df = pd.DataFrame(columns=['method', 'mic', 'distance', 'error', 'algorithm'])

distances_grid = np.arange(min(df_dist), max(df_dist))
distances = df_dist
mic_indices = range(4)
n_mics = len(mic_indices)

distance_estimators = {}

with progressbar.ProgressBar(max_value=len(distances)) as p:
    for i_d, distance in enumerate(distances):
        
        distance_estimators = {
            k: DistanceEstimator() for k in method_dict.keys()
        }
        
        p.update(i_d)
        for i_mic, mic_idx in enumerate(mic_indices):
            for method, normalize_method in method_dict.items():
                #df = results.loc[results.normalization == method]
                
                if method == "theoretical": 
                    freqs = wall_detector.df[wall_detector.df.distance==distance].frequency.unique()
                    slice_exp = get_freq_slice_theory(freqs, distance, chosen_mics=[mic_idx])
                    slice_exp = slice_exp[:, 0]
                    std = 1
                else:
                    #slice_exp, freqs, stds = wall_detector.get_frequency_slice(
                    #    distance, normalize_method=normalize_method, mics=[mic_idx])
                    slice_exp, freqs, stds = wall_detector.get_frequency_slice_fixed(
                        df_freq, distance, normalize_method=normalize_method, mics=[mic_idx])
                    slice_exp = slice_exp[0]
                    std = stds[0]
                    
                    #matrix_exp = results.loc[results.normalization==method].iloc[0].matrix
                    #slice_exp = matrix_exp[i_mic, :, i_d]
                    #freqs = df_freq
                #slice_exp = fill_nans(slice_exp, freqs)

                proba_cost = get_probability_cost(
                    slice_exp, freqs, 
                    distances_grid, mic_idx=mic_idx
                )
                distances_bayes, proba_bayes, diff_bayes = get_probability_bayes(
                    slice_exp,
                    freqs, 
                    mic_idx=mic_idx,
                    distance_range=[min(distances_grid), max(distances_grid)],
                    sigma=std
                )
                
                distance_estimators[method].add_distribution(diff_bayes * 1e-2, proba_bayes, mic_idx)

                for algo, proba, distances_here in zip(
                    ["cost", "bayes"],
                    [proba_cost, proba_bayes],
                    [distances_grid, distances_bayes],
                ):
                    d = get_estimate(distances_here, proba)
                    err_df.loc[len(err_df), :] = {
                        'error': d - distance,
                        'mic': mic_idx,
                        'distance': distance,
                        'method': method, 
                        'algorithm': algo
                    }
                    
        for method, distance_estimator in distance_estimators.items():
            distances_here, proba = distance_estimator.get_distance_distribution()
            d = get_estimate(distances_here, proba)
            err_df.loc[len(err_df), :] = {
                'error': d - distance,
                'mic': -1,
                'distance': distance,
                'method': method, 
                'algorithm': 'bayes-combination' 
            }

In [ ]:
from plotting_tools import plot_performance
titles = {
    'cost': 'optimization-based method',
    'bayes': 'FFT-based method',
    'bayes-combination': 'combination'
}
err_df = err_df.apply(pd.to_numeric, axis=0, errors='ignore')

fname = f"{exp_name}_{motors}"
for key, df in err_df.groupby("algorithm"):
    for mic, df_mic in df.groupby("mic"):
        absolute_err = pd.pivot_table(df_mic, index="distance", values="error", columns="method")
        fig, axs = plot_performance(absolute_err, xs=distances, xlabel="distance [cm]", ylabel="absolute error [cm]")
        axs[0, 0].grid()
        #axs[0, 0].set_title(f'mic{mic}, {titles[key]}')
        #axs[0, 1].set_title(f'mic{mic}, {titles[key]}')
        fig.suptitle(f'mic{mic}, {titles[key]}')
        axs[0, 1].set_xlim(-1, max(distances))
        
        fname_here = f"plots/{fname}_{key}_mic{mic}_frequency_performance.png"
        save_fig(fig, fname_here)

In [ ]:
from inference import get_probability_cost, get_probability_bayes
from simulation import get_freq_slice_theory
from matplotlib.ticker import FormatStrFormatter

mic_idx = 1

chosen_methods = ["raw", "calibration-median", "theoretical"]
#chosen_methods = method_dict.keys()
plot_combis = [{
    'distance': 11,
    'algorithms': ["cost", "bayes"]
}]
    
for plot_combi in plot_combis: 
    distance = plot_combi.get('distance')
    algorithms = plot_combi.get('algorithms')
    fname = f"{exp_name}_{motors}_{distance:.0f}cm"
    
    fig_slice, axs_slice = plt.subplots(len(chosen_methods), n_mics, sharex=True, sharey=True)
    fig_slice.set_size_inches(3*n_mics, 2*len(chosen_methods))
    
    fig_algos = {}
    axs_algos = {}
    for algo in algorithms:
        fig_algo, axs_algo = plt.subplots(len(chosen_methods), n_mics, sharex=True, sharey='row')
        fig_algo.set_size_inches(3*n_mics, 2*len(chosen_methods))
        fig_algos[algo] = fig_algo
        axs_algos[algo] = axs_algo
    
    for i_mic, mic_idx in enumerate(mic_indices):
        for i_method, method in enumerate(chosen_methods):
            df = results.loc[results.normalization == method]
            if method == "theoretical": 
                freqs = np.sort(wall_detector.df[wall_detector.df.distance==distance].frequency.unique())
                slice_exp = get_freq_slice_theory(freqs, distance)
                slice_exp = slice_exp.T
                stds = [1]*slice_exp.shape[0]
            else:
                normalize_method = method_dict[method]
                slice_exp, freqs, stds = wall_detector.get_frequency_slice(
                    distance, normalize_method=normalize_method)

            slice_exp = fill_nans(slice_exp, freqs)
            slice_exp = slice_exp[mic_idx]

            # doing this here for plotting reasons. Doesn't change performance as
            # this is done again in get_probability_cost
            slice_exp -= np.nanmean(slice_exp)
            slice_exp /= np.nanstd(slice_exp)

            axs_slice[i_method, i_mic].plot(freqs, slice_exp, label=method, color=f"C{i_method}")
            axs_slice[i_method, i_mic].set_xlim(min_freq, max_freq)
            axs_slice[i_method, 0].set_ylabel(f'{method}')
            axs_slice[0, i_mic].set_title(f"mic{mic_idx}")
            
            for algo in algorithms:
                axs_algo = axs_algos[algo]
                
                if algo == "cost":
                    proba = get_probability_cost(
                        slice_exp, freqs, 
                        distances_grid, mic_idx=mic_idx
                    )
                    distances_here = distances_grid
                elif algo == "bayes":
                    distances_here, proba, diff = get_probability_bayes(
                        slice_exp,
                        freqs, 
                        mic_idx=mic_idx,
                        distance_range=[min(distances_grid), max(distances_grid)],
                        sigma=stds[mic_idx]
                    )

                d_idx = np.argmax(proba)
                d = distances_here[d_idx]

                axs_algo[i_method, i_mic].semilogy(distances_here, proba, color=f"C{i_method}")
                axs_algo[i_method, i_mic].axvline(x=d, color=f"C{i_method}", label=f'estimate')
                axs_algo[i_method, i_mic].axvline(x=distance, color="black", ls=":", label=f'real: {distance:.0f}cm')
                axs_algo[i_method, i_mic].set_xlim(min(distances_here)-1, max(distances_here)+1)

                axs_algo[0, i_mic].set_title(f"mic{mic_idx}")
                axs_algo[i_method, 0].set_ylabel(f'{method}')

    fname_here = f'plots/{fname}_slice.png'
    save_fig(fig_slice, fname_here)

    for algo in algorithms:
        axs_algos[algo][-1, -1].legend(loc='upper right')
        fname_here = f'plots/{fname}_{algo}.png'
        save_fig(fig_algos[algo], fname_here)

# Calibration study

In [ ]:
from calibration import fit_distance_slice
from simulation import get_amplitude_function

fname = f'{exp_name}_{motors}'

all_distances = wall_detector.df.distance.unique()

fitting_results = pd.DataFrame(
    columns=["frequency", "mic", "absorption", "gains", "offset", "method", "limit_distance"]
)

# found from below
plot_tuples = [(0, 3093), (0, 3218), (3, 3734)]

for i, (frequency, df_freq) in enumerate(wall_detector.df.groupby('frequency')):
    
    # does not make a difference
    # df_here = df_here[df_here.magnitude > 2]
    distances = df_freq.distance.unique()
    if len(distances) < len(all_distances): # only plot distances with "full coverage"
        continue
    
    slices_median, distances_median_all, mics, stds = wall_detector.get_distance_slice(frequency, method=np.nanmedian)
    
    # TODO: these don't make sense because they model the full oscillation
    #print('stds:', stds)
    
    # global
    coeffs_raw_glob, *_ =  wall_detector.fit_to_raw(frequency)
    coeffs_median_glob, *_ =  wall_detector.fit_to_median(frequency)
    
    fitting_results.loc[len(fitting_results), :] = dict(
        frequency=frequency,
        mic=-1,
        absorption=coeffs_raw_glob[0],
        gains=coeffs_raw_glob[2],
        offset=coeffs_raw_glob[1],
        method="one-shot raw",
    )
    fitting_results.loc[len(fitting_results), :] = dict(
        frequency=frequency,
        mic=-1,
        absorption=coeffs_median_glob[0],
        gains=coeffs_median_glob[2],
        offset=coeffs_median_glob[1],
        method="one-shot median",
    )
        
    for i_mic, (mic, df_here) in enumerate(df_freq.groupby('mic')):
        mic_idx = int(mic)
            
        coeffs_raw, distances_raw, fit_raw, cost_raw =  wall_detector.fit_to_raw(frequency, mic_idx)
        coeffs_median, fit_median, distances_median, cost_median =  wall_detector.fit_to_median(frequency, mic_idx)
        
        # find the sigma for this frequency (per distance)
        alpha, phase, gain = coeffs_raw
        std_series = df_here.groupby('distance').magnitude.std()
        std_average = np.mean(std_series.values)
        amps = get_amplitude_function(std_series.index, 
                                      gain, 
                                      alpha, mic_idx)
        valid_distances = std_series.index[amps >= std_average]
        limit_distance = valid_distances[-1] if len(valid_distances) else 0
        
        fitting_results.loc[len(fitting_results), :] = dict(
            frequency=frequency,
            mic=mic_idx,
            absorption=coeffs_median[0],
            gains=coeffs_median[2],
            offset=coeffs_median[1],
            method="median",
        )
        fitting_results.loc[len(fitting_results), :] = dict(
            frequency=frequency,
            mic=mic_idx,
            absorption=coeffs_raw[0],
            gains=coeffs_raw[2],
            offset=coeffs_raw[1],
            method="raw",
            limit_distance=limit_distance,
        )
        
        if (mic_idx, frequency) not in plot_tuples:
            continue
            
        print(f'plotting: mic {mic}, frequency {frequency}, alpha={alpha:.2f}, gain={gain:.2f}')
            
        label = f"{frequency:.0f}Hz"
        fig, axs = plt.subplots(1, 2)
        fig.set_size_inches(10, 5)
        
        ax_fit, ax_freq = axs
        
        for d, series in df_here.groupby('distance').magnitude: 
            ax_fit.scatter([d]*len(series), series.values, color='C0', s=5.0)
        ax_fit.scatter([], [], color='C0', s=2.0, label='raw')
        ax_fit.plot(distances_raw, fit_raw, color='C2', label='fit to raw')
        
        ax_fit.plot(distances_median_all, slices_median[i_mic], color='C1', label='median')
        ax_fit.plot(distances_median, fit_median, color='C3', ls=':', label='fit to median')
        
        ax_fit.set_title(label)
        ax_fit.set_ylabel('amplitude [-]')
        ax_fit.set_xlabel('distance [cm]')
        ax_fit.legend(loc='upper right')
        
        ax_freq.scatter(std_series.index, std_series.values, label=f'std', color=f'C{i}')
        #ax_freq.axhline(stds[i_mic], label=f'average std', color=f'C{i}')
        ax_freq.axhline(std_average, label=f'average std', color=f'C{i}')
        ax_freq.semilogy(std_series.index, amps*0.5, ls=':', color=f'C{i}')
        ax_freq.semilogy(std_series.index, amps, ls=':', color=f'C{i}', label=f'amplitude')
        ax_freq.semilogy(std_series.index, amps*1.5, ls=':', color=f'C{i}')
        ax_freq.set_title(label)
        ax_freq.legend(loc='upper right')
        ax_freq.set_xlabel('distance [cm]')
        ax_freq.set_ylabel('magnitude vs. std [-]')
        
        fname_here = f'plots/{fname}_fitting_{frequency:.0f}_{mic_idx}.png'
        save_fig(fig, fname_here)

In [ ]:
fig = plt.figure()
fig.set_size_inches(5, 5)
df = fitting_results.loc[fitting_results.method=='raw']
for mic, df_mic in df.groupby('mic'):
    plt.plot(df_mic.frequency, df_mic.limit_distance, color=f'C{mic}', label=f'mic{mic}')
    
    success = df_mic.loc[df_mic.limit_distance == df_mic.limit_distance.max()].frequency.values
    fail = df_mic.loc[df_mic.limit_distance < df_mic.limit_distance.max()].frequency.values
    print(f'success mic{mic}:', success)
    print(f'fail mic{mic}:', fail)
plt.xlabel('frequency [Hz]')
plt.ylabel('limit distance [cm]')
plt.legend()
fname_here = f'plots/{fname}_limit_distance.png'
save_fig(fig, fname_here)


In [ ]:
import seaborn as sns
color_palette = "tab10"
extra_kwargs = {
    "linewidth": 0,
    "palette": color_palette,
    "style": "method",
    "hue": "mic",
    "data": fitting_results,
    "x": "frequency",
}
palette = sns.palettes.color_palette(color_palette)
fitting_results = fitting_results.apply(pd.to_numeric, errors="ignore", axis=0)
plt.figure()
sns.scatterplot(
    y="gains", **extra_kwargs,
)

plt.figure()
sns.scatterplot(y="absorption", **extra_kwargs)

plt.figure()
sns.scatterplot(y="offset", **extra_kwargs)
plt.ylabel("offset [cm]")

ls = {"one-shot raw": "-", "median": ":", "one-shot median": "--", "raw": "-."}
for method, df in fitting_results.groupby("method"):
    medians = df.groupby(["mic"]).offset.median()
    [plt.axhline(m, color=palette[i], ls=ls[method]) for i, m in enumerate(medians)]
    plt.plot([], [], color="black", ls=ls[method], label=method)
plt.legend(loc="upper left", bbox_to_anchor=[1, 1])

